We will be using open source libraries and large language models (llama index and Mistral-7b) to perform Q&A with PDF articles. LlamaIndex is a data framework for connecting custom data sources to LLMs.

We will be running the code using Google Colab. Please make sure that the T4 GPU instance of Colab notebook is activated via the notebook settings before proceeding.

Code is based on this tutorial: https://www.youtube.com/watch?v=1mH1BvBJCl0. See also llama index documentation here: https://docs.llamaindex.ai/en/stable/examples/llm/llama_2_llama_cpp.html

In [ ]:
!pip install -q pypdf
!pip install -q python-dotenv
!pip install -q transformers
!pip install -q llama-index
!pip install -q sentence-transformers
!pip install -q langchain
# Ask Llama to use GPU
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1

In [ ]:
# Logging
import logging
import sys
# Format the response from LLM so they display properly
from IPython.display import Markdown, display

# import LLama index related functions
import torch
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt  # pass in messages_to_prompt and completion_to_prompt functions to help format the model inputs.

# Import embedding libraries and functions
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding

Turn on logging

In [ ]:
# Make sure logging is turned
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

Load in your Mistral model here. We will be using the fine-tuned, quantized Mistral 7b model for everything

In [ ]:
llm = LlamaCPP(
    # You can pass in the URL to a GGUF model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

total size (MB): 4368.44


4167it [00:22, 184.10it/s]                          
AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


Load your embedding model and specify the directory containing your PDFs

In [ ]:
# Change embedding model to suit your needs. A list of embedding models from Hugging Face can be found here: https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=trending
embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)

# Place your PDF articles that will be used for Q&A in a folder called 'Data'. The PDF reader will extract the PDF into text
documents = SimpleDirectoryReader("/content/Data/").load_data()

Specify service context. The default model for llama_index is from OpenAI, here we specify that we are using our custom LLM and embedding model. Chunk size tells us the size of the context that will be sent to the model.

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

# Initiate the vector store
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

# Perform your query

The PDF I uploaded was my scientific paper on FtsZ dynamics. It is quite technical and could be challenging for the model.

In cases where you want to use the top k chunks refer to documentation here: https://docs.llamaindex.ai/en/stable/understanding/querying/querying.html#customizing-the-stages-of-querying

In [ ]:
%%time
query_engine = index.as_query_engine()
response = query_engine.query("What are the similarities between Z ring structures in S. aureus and B. subtilis?")
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


<b> Based on the provided context information, it appears that there are some similarities between Z ring structures in S. aureus and B. subtilis. When imaged in the axial plane, S. aureus Z rings were almost identical to typical 3D-SIM images of B. subtilis Z rings acquired in the same orientation (Figure 3E, Movie S2). This suggests that both organisms have similar overall structures for their Z rings. However, when imaged in the lateral plane, there are differences between the two organisms. In S. aureus, there was a heterogeneous, bead-like distribution of FtsZ throughout the entire Z ring, with visible "gaps" often observed within the ring (Figure 3D and 3F). On the other hand, in B. subtilis, the lateral plane imaging showed a more uniform distribution of FtsZ within the Z ring. Additionally, the concentration of FtsZ-GFP within the ring can typically vary by up to 4-fold in S. aureus (Figure 3G), while in B. subtilis, there is no mention of such variation. Therefore, while there are</b>

CPU times: user 28.5 s, sys: 32 ms, total: 28.6 s
Wall time: 28.7 s
